In [1]:
import os
from datetime import datetime
import logging
import json

import pandas as pd
import tifffile as tiff

from flame import FLAMEImage
from flame.error import FLAMEImageError

In [2]:
INPUT_DIREC = "/mnt/d/data/raw"
OUTPUT_DIREC = "/mnt/d/data/processed"
DATASET_DIREC = os.path.join(os.getcwd(), "datasets")
DS_TYPE = "denoising"
INPUT_N_FRAMES = 7
OUTPUT_N_FRAMES = 40

In [3]:
logger = logging.getLogger("main")
logging.basicConfig(
    filename=f"{datetime.now().strftime('%Y%m%d-%H%M%S')}_logger.log",
    encoding="utf-8",
    level=logging.DEBUG
)

### Find images to be used in the dataset

In [4]:
IMAGE_INDEX = pd.read_csv(os.path.join(DATASET_DIREC, "raw_image_index.csv"))
IMAGE_INDEX.head()

,id,image
0,0,S268_250409_CAREtraining_PL/CAREtraining_ExViv...
1,1,S268_250409_CAREtraining_PL/CAREtraining_ExViv...
2,2,S268_250409_CAREtraining_PL/CAREtraining_ExViv...
3,3,S268_250409_CAREtraining_PL/CAREtraining_ExViv...
4,4,S268_250409_CAREtraining_PL/CAREtraining_ExViv...


In [5]:
this_ds = {}
for idx, relpath in zip(IMAGE_INDEX['id'], IMAGE_INDEX['image']):
    this_impath = os.path.join(INPUT_DIREC, relpath)
    if os.path.isfile(this_impath):
        logger.info(f"Found image of id {idx} at {this_impath}")
        try:
            this_image = FLAMEImage(this_impath, "tileData.txt")
        except FLAMEImageError as e:
            logger.error(f"Could not initialize image of id {idx} at {this_impath}")
            continue
        this_ds[idx] = this_impath
    else:
        logger.error(f"Could not find image of id {idx} at {this_impath}")

In [6]:
logger.info(f"Found {len(this_ds)} images for dataset")
print(f"Found {len(this_ds)} images for dataset")
if len(this_ds) < 1:
    logger.error(f"No valid images were found in {INPUT_DIREC}")
    raise Exception(f"No valid images were found in {INPUT_DIREC}")

Found 3 images for dataset


### Documenting Dataset

In [7]:
DS_NAME = f"{datetime.now().strftime('%Y%m%d')}_{len(this_ds)}I_{DS_TYPE}_{INPUT_N_FRAMES}to{OUTPUT_N_FRAMES}F"

In [8]:
dataset_json = {
    "FLAME_Dataset": {
        "name": DS_NAME,
        "type": DS_TYPE,
        "input_frames": INPUT_N_FRAMES,
        "output_frames": OUTPUT_N_FRAMES,
        "image_ids": list(this_ds.keys())
    }
}

In [9]:
json_path = os.path.join(DATASET_DIREC, f"{DS_NAME}.json")
json.dump(dataset_json, open(json_path, "w+"))
logger.info(f"Saving dataset config JSON to {json_path}")

### Creating Dataset

In [10]:
DS_OUTPUT_DIREC = os.path.join(OUTPUT_DIREC, DS_NAME)
os.makedirs(DS_OUTPUT_DIREC, exist_ok=True)
logger.info(f"Created dataset output directory at {DS_OUTPUT_DIREC}")

In [11]:
FLAME_Images = []
for idx, impath in this_ds.items():
    this_image = FLAMEImage(impath, "tileData.txt")
    this_image.openImage() # load image data into memory first

    input_frames_path = os.path.join(DS_OUTPUT_DIREC, f"id{idx}_frames{INPUT_N_FRAMES}.tif")
    input_frames = this_image.get_frames(0, INPUT_N_FRAMES)
    logger.info(f"Saving {input_frames_path}...")
    tiff.imwrite(input_frames_path, input_frames)
    
    output_frames_path = os.path.join(DS_OUTPUT_DIREC, f"id{idx}_frames{OUTPUT_N_FRAMES}.tif")
    output_frames = this_image.get_frames(0, OUTPUT_N_FRAMES)
    logger.info(f"Saving {output_frames_path}...")
    tiff.imwrite(output_frames_path, output_frames)

    this_image.closeImage() # close image and force garbage collection for memory management
    FLAME_Images.append(this_image)

In [12]:
an_image = FLAME_Images[0]
an_image.openImage()

In [13]:
an_image.tileData.framesPerTile

1

In [14]:
an_image.impath

'/mnt/d/data/raw/S268_250409_CAREtraining_PL/CAREtraining_ExVivo_scalp/Mosaic02_1x7_FOV1100_z65_3Ch/Im_00007.tif'